In [ ]:
from typing import List, Union
import logging
import os
import sys
import joblib
import fire
import fairseq
import soundfile as sf
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
from einops import rearrange
import re
import numpy as np
from functools import partial
import torch.multiprocessing as mp
import torchaudio
import glob
import tqdm
import argparse
from torchaudio.functional import resample

In [ ]:
from speech2unit import Speech2Unit

In [ ]:
# download the files from here : https://drive.google.com/drive/folders/1bKjqa9N_AAM5PTqL4Z-5RxQHimTyCdJO
s2u = Speech2Unit(
        ckpt_path=ckpt_path, #mhubert_base_vp_en_es_fr_it3.pt
        km_path = km_path   #mhubert_base_vp_en_es_fr_it3_L11_km1000.bin
    )

In [ ]:
units = s2u('example wav file as input')
print(units)

In [ ]:
def get_transcript(file_path):
    with open(file_path, 'r') as file:
        return file.readline()
get_transcript('/media/iiit/Karvalo/zuhair/Proj-multimodal/dailytalk/data/0/0_1_d0.txt')

In [ ]:
import os
from tqdm import tqdm
import pandas as pd

# Define the directory containing the audio files
audio_dir = "/media/iiit/Karvalo/zuhair/Proj-multimodal/dailytalk/data"

# Custom sorting function to sort files based on a specific pattern
def custom_sort(file):
    parts = file.split('_')
    return int(parts[0]), int(parts[1]), parts[2]  # Sort by the first and second parts as integers and the third part as a string

# Function to split the list of files into .txt and .wav lists
def split_files_by_extension(files):
    txt_files = [file for file in files if file.endswith('.txt')]
    wav_files = [file for file in files if file.endswith('.wav')]
    return txt_files, wav_files

speech_units = []  # List to store the formatted conversation units

# Get a list of all folders in the audio directory and sort them numerically
files = os.listdir(audio_dir)
files.sort(key=lambda x: int(x))

# Loop through each folder (conversation) in the directory
for n, folder in enumerate(tqdm(files, desc="Processing")):
    conversation_dir = audio_dir + '/' + folder  # Path to the current conversation folder
    files = os.listdir(conversation_dir)  # List all files in the current conversation folder
    files = sorted(files, key=custom_sort)  # Sort the files using the custom_sort function

    # Split the files into text and audio lists
    txt_files, wav_files = split_files_by_extension(files)

    file_units = []  # List to store the units for this conversation
    for N, file in enumerate(wav_files):
        if (N+1) % 2 == 1:  # Odd files correspond to speaker 'A'
            speaker = 'A'
            units = '<A><sp>' + s2u(conversation_dir + f'/{file}') + '</sp></A>'  # Process and format the audio file for speaker A
        else:  # Even files correspond to speaker 'B'
            speaker = 'B'
            units = '<B><txt>' + get_transcript(conversation_dir + '/' + file.split('.')[0] + '.txt') + '</txt></B>'  # Process and format the transcript for speaker B
        file_units.append(units)  # Add the formatted unit to the list for this conversation
    
    conversation = ' '.join(file_units)  # Combine all units for this conversation into a single string
    speech_units.append(conversation)  # Add the conversation string to the main list

# Create a DataFrame from the collected conversations
df = pd.DataFrame({
    'hubert_units': speech_units
})

# Save the DataFrame to a CSV file
df.to_csv('/media/iiit/Karvalo/zuhair/Proj-multimodal/dailytalk/QandA_100.csv')
